In [8]:
import cv2
import logging
import numpy
import typing
from utils import (
    create_folder, 
    get_logger, 
    debounce, 
    throttle, 
    get_homography_martix, 
    select_points)

logger: logging.Logger = get_logger(__name__)



In [9]:
image1 = cv2.imread('im011/im01.jpg')
image2 = cv2.imread('im011/im02.jpg')
image2 = cv2.resize(src=image2, dsize=(image1.shape[1], image1.shape[0]))

points_im01 = []
points_im02 = []
cv2.namedWindow('im011/im01.jpg')
cv2.setMouseCallback('im011/im01.jpg', 
                     select_points,
                    {"image": image1,
                     "point_list": points_im01})
cv2.namedWindow('im011/im02.jpg')
cv2.setMouseCallback('im011/im02.jpg', 
                     select_points,
                    {"image": image2,
                     "point_list": points_im02})
while True:
    # time.sleep(200)
    cv2.imshow('im011/im01.jpg', image1)
    cv2.imshow('im011/im02.jpg', image2)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()
cv2.waitKey(1)


-1

In [21]:
logger.info(f"points_im01:{points_im01}")
logger.info(f"points_im02:{points_im02}")

image1 = cv2.imread('im011/im01.jpg')
image2 = cv2.imread('im011/im02.jpg')
image2 = cv2.resize(src=image2, dsize=(image1.shape[1], image1.shape[0]))
logger.info(f"image1.shape:{image1.shape}")
logger.info(f"image2.shape:{image2.shape}")

H_2_to_1 = get_homography_martix(points_1=points_im02, points_2=points_im01)
image2_height, image2_width, _ = image2.shape
image1_height, image1_width, _ = image1.shape

custom_result1 = numpy.zeros(shape=[image1.shape[0] + image1.shape[0], image1.shape[1] + image1.shape[1] , image1.shape[2]], 
                            dtype=image1.dtype)
result_height, result_width, _ = custom_result1.shape
delta_y = int(result_height/ 4)
delta_x = int(result_width/ 4)

for y in range(image2_height):
    for x in range(image2_width):
        p = numpy.dot(H_2_to_1, numpy.array([x, y, 1]))
        p = p / p[2]
        target_x, target_y = int(p[0]), int(p[1]) + delta_y
        if 0 <= target_x < result_width and 0 <= target_y < result_height:
            custom_result1[target_y, target_x] = image2[y, x]
# the reflection is not successive- >
# therefore there are some internal empty points
# black_point_mask = (custom_result1 == 0).astype(numpy.uint8)
# black_point_mask = cv2.cvtColor(src=black_point_mask, code=cv2.COLOR_BGR2GRAY)
# custom_result1 = cv2.inpaint(src=black_point_mask, 
#                              inpaintMask=black_point_mask, 
#                              inpaintRadius=5, 
#                              flags=cv2.INPAINT_TELEA)
cv2.imwrite('custom_result1.png', custom_result1)

custom_result2 = numpy.zeros(shape=[image2.shape[0] + image2.shape[0], image2.shape[1] + image2.shape[1] , image2.shape[2]], 
                            dtype=image2.dtype)

result_height, result_width, _ = custom_result2.shape
delta_y = int(result_height/ 4)
delta_x = int(result_width/ 4)
for y in range(image1_height):
    for x in range(image1_width):
        # target_y, target_x = int(y * 2), int(x * 2)
        target_y, target_x = y + delta_y, x
        if 0 <= target_x < result_width and 0 <= target_y < result_height:
            custom_result2[target_y, target_x] = image1[y, x]


result = cv2.addWeighted(src1=custom_result1, 
                         alpha=0.5, 
                         src2=custom_result2, 
                         beta=0.5, 
                         gamma=0)

cv2.imwrite('part3_custom_homography_image2.png', result)
# combo_result = (image1 + custom_result) / 2
# cv2.imwrite('part3_combo.png', combo_result)




2023-10-02 01:52:51,648 - __main__ - INFO - points_im01:[(615, 257), (322, 228), (329, 421), (614, 465)]
2023-10-02 01:52:51,648 - __main__ - INFO - points_im01:[(615, 257), (322, 228), (329, 421), (614, 465)]
2023-10-02 01:52:51,648 - __main__ - INFO - points_im01:[(615, 257), (322, 228), (329, 421), (614, 465)]
INFO:__main__:points_im01:[(615, 257), (322, 228), (329, 421), (614, 465)]
2023-10-02 01:52:51,660 - __main__ - INFO - points_im02:[(320, 283), (24, 246), (21, 473), (316, 469)]
2023-10-02 01:52:51,660 - __main__ - INFO - points_im02:[(320, 283), (24, 246), (21, 473), (316, 469)]
2023-10-02 01:52:51,660 - __main__ - INFO - points_im02:[(320, 283), (24, 246), (21, 473), (316, 469)]
INFO:__main__:points_im02:[(320, 283), (24, 246), (21, 473), (316, 469)]
2023-10-02 01:52:51,703 - __main__ - INFO - image1.shape:(480, 640, 3)
2023-10-02 01:52:51,703 - __main__ - INFO - image1.shape:(480, 640, 3)
2023-10-02 01:52:51,703 - __main__ - INFO - image1.shape:(480, 640, 3)
INFO:__main__:i

True

: 